<a href="https://colab.research.google.com/github/namiyousef/argument-mining/blob/develop/experiments/yousef/GPU_otpimisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -- env setup
import os
import gc

!python3.7 -m pip install git+https://github.com/namiyousef/colab-utils.git
from colabtools.utils import get_gpu_utilization, mount_drive, install_private_library

drive_path = mount_drive()
project_path = os.path.join(drive_path, 'argument-mining')
development_dir = os.path.join(drive_path, 'argument-mining/experiments/yousef')

install_private_library(os.path.join(project_path, 'data/github_config.json'), 'argument-mining')

# -- public imports
import sys
from transformers import BigBirdTokenizer, AutoTokenizer, AutoModelForTokenClassification, AutoConfig
import pandas as pd
from torch.utils.data import DataLoader
import torch

# -- private imports
from argminer.data import KaggleDataset

# -- dev imports
# add working package to system path
sys.path.append(development_dir)
%load_ext autoreload
%autoreload 2
from python_files.data import create_labels_doc_level
from python_files.run import train_longformer

  Cloning https://github.com/namiyousef/colab-utils.git to /tmp/pip-req-build-zdd9jwpb
  Running command git clone -q https://github.com/namiyousef/colab-utils.git /tmp/pip-req-build-zdd9jwpb
Google Drive import successful.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mount successful.
Import of ArgMiner successful
CUDA device detected. Using GPU...


In [31]:
MODEL_NAME = 'allenai/longformer-base-4096'
print(f'GPU usage: {get_gpu_utilization()}')
try:
  ma = torch.cuda.memory_allocated()
  print(ma)

  mc = torch.cuda.memory_reserved()
  print(mc)

  del tokenizer, model, optimizer, dataset, train_loader, df_kaggle, config_model
  gc.collect()
  torch.cuda.empty_cache()
  ma = torch.cuda.memory_allocated()
  print(ma)

  mc = torch.cuda.memory_cached()
  print(mc)
  print(f'GPU usage after deletion: {get_gpu_utilization()}')
except:
  tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, add_prefix_space=True)

  config_model = AutoConfig.from_pretrained(MODEL_NAME) 
  config_model.num_labels = 15

  model = AutoModelForTokenClassification.from_pretrained(MODEL_NAME, config=config_model)
  optimizer = torch.optim.Adam(params=model.parameters())

  df_train = pd.read_csv(os.path.join(project_path, 'data/kaggle/test.csv'))
  df_kaggle = df_train[['text', 'labels']].copy(); del df_train
  df_kaggle.labels = df_kaggle.labels.apply(lambda x: [int(num) for num in x[1:-1].split(', ')])
  df_kaggle.head(), df_kaggle.info()
  
  dataset = KaggleDataset(df_kaggle, tokenizer, 512)
  train_loader = DataLoader(dataset, shuffle=True, batch_size=1)

  print(f'GPU usage after creation: {get_gpu_utilization()}')
  torch.backends.cudnn.benchmark = True
  torch.backends.cudnn.enabled = True
  print(f'GPU mem: {get_gpu_utilization()}')
  model.to('cuda:0')
  print(f'GPU mem after model sent: {get_gpu_utilization()}')
  for i, (inputs, targets) in enumerate(train_loader):
      inputs = {key: val.to('cuda:0') for key, val in inputs.items()}
      targets = targets.to('cuda:0')

      optimizer.zero_grad()
      loss, outputs = model(labels=targets, **inputs, return_dict=False)
      print(f'GPU mem modelling batch {i+1}: {get_gpu_utilization()}')
      del targets, inputs, loss, outputs
      gc.collect()
      torch.cuda.empty_cache()
      print(f'GPU mem batch {i+1}: {get_gpu_utilization()}')
      break
  model.to('cpu')



GPU usage: 1330
0
0


Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForTokenClassification: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing LongformerForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LongformerForTokenClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15594 entries, 0 to 15593
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    15594 non-null  object
 1   labels  15594 non-null  object
dtypes: object(2)
memory usage: 243.8+ KB
GPU usage after creation: 1330
GPU mem: 1330
GPU mem after model sent: 1954
GPU mem modelling batch 1: 2806
GPU mem batch 1: 1954


In [3]:
!nvidia-smi --gpu-reset

GPU 00000000:00:04.0 is currently in use by another process.

1 device is currently being used by one or more other processes (e.g., Fabric Manager, CUDA application, graphics application such as an X server, or a monitoring application such as another instance of nvidia-smi). Please first kill all processes using this device and all compute applications running in the system.


In [4]:
!sudo fuser -v /dev/nvidia*

                     USER        PID ACCESS COMMAND
/dev/nvidia0:        root      14034 F...m python3
/dev/nvidiactl:      root      14034 F...m python3
/dev/nvidia-uvm:     root      14034 F...m python3


In [5]:
!sudo fuser -v /dev/nvidia*

                     USER        PID ACCESS COMMAND
/dev/nvidia0:        root      14034 F...m python3
/dev/nvidiactl:      root      14034 F...m python3
/dev/nvidia-uvm:     root      14034 F...m python3


In [6]:
!nvidia-smi

Mon Mar  7 23:34:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    27W /  70W |   1954MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
import argminer